# INFO 2950 Discussion Week 3

For our group time on Friday we will be going through a tutorial on web scraping together.

**Make sure to complete and save this notebook before next Friday's section!** We will use the completed version for an activity on pushing to GitHub next week.




## What is web scraping?

Web scraping is the process of creating structured data frames readable by computers from less-structured web pages that are intended to be read by humans.

## Do you need to scrape?

Scraping is difficult, error-prone, and may get you in legal or ethical trouble. Always check if the information you are seeking already exists in a nicer format. Many companies also have APIs or other infrastructure dedicated to serving automated requests.

## Ethics of scraping

Scraping is the aspect of data science most likely to get you into trouble. Legally, there are two main concerns: copyright violation if you are making copies of content that is protected by copyright, and terms-of-service violation. Many sites will also attempt to block scrapers. Acting to avoid these blocks may constitute hacking.

In addition to legal problems, there are ethical problems.

* Respect other people's hard work. Collecting information for a cool website is difficult and takes a long time and careful preparation. Is it fair to take all that work and use it for your purposes?
* Respect other people's businesses. People feed their families by collecting and sharing information, for example through ad revenue. They may be unhappy if they think you are trying to steal their work and prevent them from monetizing it. Websites also sometimes pay for bandwidth and cloud services, and you don't want to take up all their resources for purposes that provide no revenue.
* Respect other people's audiences. Don't [DDOS](https://en.wikipedia.org/wiki/Denial-of-service_attack) a website! Keep requests spaced to about 2/second at most. Web hits don't take a lot of resources, but it's easy to flood a site with requests, locking others out.

## Process

Find a page that contains the info you want. Some older pages are static, but most of what you will see today are views of databases. For example, Amazon has a database of products, Letterboxd has a database of movies, and Netflix has a database of shows. Each page either lists entities in the database or gives information about specific entities.

You would prefer to get access directly to the database, and in some cases you can. But for whatever reason sometimes the web page is all you can get. This process can be difficult because web pages are optimized for people to look at, not for computers to operate on. But it is simpler because in this case a computer has to generate a page *from* a database record and a web browser has to know how to display the elements on the page so that humans can read them. These constraints mean that there are regularities in how specific pieces of information are shown in  HTML, and if you can figure them out, you can reverse the page generation process and get back to something that looks more like a database.

How do you know which pages to get? Often an index or search result page leads to individual pages with more detailed information.


### Web scraping in Python

The [`requests`](https://requests.readthedocs.io/en/latest/) library allows you to download files from the web. It is much easier to use than `urllib`. You can use the `requests` library to get information from web pages so that you can save them to files or analyze their data in python.

After using `requests` to access web data, you'll use [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to parse that information, organized in HTML. `BeautifulSoup` makes querying a tree of tags and their attributes much easier than trying to parse HTML from scratch. You'll need to spend some time looking at the target web page and finding the combination of tag names and classes you're interested in, but `BeautifulSoup` can help access that information once you know what you need.



## Installation
As usual, to use these new modules you'll need to first **install `BeautifulSoup` and `requests` to your virtual environment for this class**. Only once you've completed that process can successfully you run the next cell of code to load in `requests`, `BeautifulSoup`, and other modules.





In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time



## In-session Example: Cornell University Website

The TA will show you [this page about Cornell University](https://www.cornell.edu/about/). This webpage has information about Cornell, including general statistics about number of students, different campus locations, and many links to other Cornell pages.

## Viewing HTML
Take a look at the HTML of the page. You can view this in a couple of different ways:

*   Right click > "View Page Source"
*   Right click > "Inspect"

**What differences do you notice between the two HTML views?**



Today, our goal is to collect and organize various stats about Cornell.

We need to put together clues from the structure of the page and the appearance of elements on the page. Appearance is usually determined by the CSS `style` property. Figuring out how to automatically find values from HTML will involve looking at HTML source, ^F searching for values that you want, and figuring out how to identify styles or containing elements. Modern web pages are long and have lots of complicated elements, many of which do not appear as visible content. Starting from the top of the document and reading through is not recommended.

You will scrape information from cached copies of "About Cornell" pages. These are locally hosted so we don't have hundreds of students hitting Cornell's website at the same time. They probably wouldn't notice, but it's more responsible to keep it local.


In [2]:
cornell_url = "https://koenecke.infosci.cornell.edu/files/info2950/About%20Cornell%20University.html"

The following code does a `GET` request to the web host for the specified filename. HTTP is the protocol used to make web requests. It has a series of "status codes" that tell you the result of request. 200 is success. Others you have probably seen: 404 is "page not found", 403 is "you do not have access". Codes starting with 3-- are often redirects. 500 means there is a bug in the server-side code.

In [3]:
cornell_result = requests.get(cornell_url)
if cornell_result.status_code != 200:
  print("something went wrong:", cornell_result.status_code, cornell_result.reason)

You're never going to get the analysis of a web page right the first time, so it's good to save a local copy of the HTML source so we don't need to hit the server again.

In [4]:
with open("cornell_about.html", "w") as writer:
  writer.write(cornell_result.text)

Here we're immediately reading the file again, but you could split these into two separate notebooks, one for downloading, one for analysis.

In [5]:
with open("cornell_about.html", "r") as reader:
  html_source = reader.read()

There are a lot of things that can go wrong when you are accessing web documents. Get in the habit of constantly adding confidence checks to make sure the state of variables is what you expect it to be.

In [6]:
# Make sure this worked
html_source[:20]

'<!DOCTYPE html>\n<htm'

Here's where we turn the HTML text from a single long string into a searchable tree of tags. `BeautifulSoup` can support different ways of parsing (including XML). Here we'll use an HTML parser.

In [7]:
page = BeautifulSoup(html_source, "html.parser")

Now that we have a structured document we can ask for specific tags. See the Beautiful Soup documentation linked at the top for more details.

In [8]:
page.title

<title>About | Cornell University</title>

If you want just the text between the HTML tags, you can use `.text`.

In [9]:
page.title.text

'About | Cornell University'

We can also find all of the instances of a given tag. Since this "About Cornell" page seems like more of a landing page, pointing to other resources, we want to use the links on this page to get us to that other linked information. What happens if we ask for all the `a` tags, which [defines a hyperlink](https://www.w3schools.com/tags/tag_a.asp)?

In [10]:
links = page.find_all("a")
print("there are", len(links), "links on the page")
links[:20]

there are 180 links on the page


[<a href="https://www.cornell.edu/about/">https://www.cornell.edu/about/</a>,
 <a href="https://www.cornell.edu/about/">current page</a>,
 <a href="http://support.google.com/websearch/bin/answer.py?hl=en&amp;p=cached&amp;answer=1687222"><span>Learn more</span>.</a>,
 <a href="http://webcache.googleusercontent.com/search?q=cache:FwQY78IHD64J:https://www.cornell.edu/about/&amp;sca_esv=562865996&amp;hl=en&amp;gl=us&amp;strip=1&amp;vwsrc=0"><span>Text-only version</span></a>,
 <a href="http://webcache.googleusercontent.com/search?q=cache:FwQY78IHD64J:https://www.cornell.edu/about/&amp;sca_esv=562865996&amp;hl=en&amp;gl=us&amp;strip=0&amp;vwsrc=1"><span>View source</span></a>,
 <a data-click-action="skip to content" data-click-category="header" href="#main" tabindex="1">Skip to content</a>,
 <a data-click-action="home" data-click-category="header" href="https://www.cornell.edu/" id="cu-brand" title="Cornell.edu Homepage">
 <div class="cu-logotype">Cornell University</div>
 </a>,
 <a class="

Whoa, that's too much! And most of these links seem to be navigation links in header menus.

Let's be more specific. Viewing the source, we can search for an interesting string (like "Facts about Cornell") and look at the HTML around it.

Inspecting the HTML we find that `a` tags can be associated with a number of class styles. The class style for "Facts about Cornell" is `link-caret`. Here's how to find all of the `a` tags with this style:

In [11]:
link_carets = page.find_all("a", {"class":"link-caret"})
print(link_carets)

[<a class="link-caret" href="https://www.cornell.edu/about/locations.cfm" title="Cornell around the globe">
  See All Locations
</a>, <a class="link-caret" href="https://irp.dpb.cornell.edu/university-factbook" title="Facts about Cornell">
  See All University Facts
</a>, <a class="link-caret" href="https://www.cornell.edu/about/timeline/" title="Cornell's Historical Timeline">
  Explore Cornell's History
</a>]


Once we find a tag, we can also access attributes of the tag, like the URL target of a link (`href`) or the text contained within the tag (`.text`).

In [12]:
for link in link_carets:
  print(link.text, link["href"])


  See All Locations
 https://www.cornell.edu/about/locations.cfm

  See All University Facts
 https://irp.dpb.cornell.edu/university-factbook

  Explore Cornell's History
 https://www.cornell.edu/about/timeline/


## Organizing structured data
Now, we want to collect different stats and facts about Cornell, so the table on the right side with "Quick Facts" seems like a great place to start. **Our goal is to recreate the "Quick Facts" table as a pandas dataframe.**

Remember, instead of reading the HTML from start to bottom, use ^F to search for the text "Undergraduate Students" (for example).

While there are different ways of collecting this tag, you really want to keep the name of the fact and its value together. For this reason, let's collect the list element as a singular unit (with tag `li` and the `stat` class style).

In [13]:
cornell_stats = page.find_all("li", {"class": "stat"})
print(len(cornell_stats))

6


Now that we have a list of every `li` (list item) with the `stat` class, we can extract the values of the list. Let's start with the first item in this new list:

In [14]:
print(cornell_stats[0].find("div", {"class": "stat-label"}).text)
print(cornell_stats[0].find("div", {"class": "stat-value"}).text)


        Undergraduate Students
      

         15,735
       


What's with all that whitespace? You can strip the extra whitespace with `.strip()`.

In [15]:
print(cornell_stats[0].find("div", {"class": "stat-label"}).text.strip())
print(cornell_stats[0].find("div", {"class": "stat-value"}).text.strip())

Undergraduate Students
15,735


Much better! Now, we can use a loop to extract the label and value from each list item.

In [16]:
for html_stat in cornell_stats:
  print(html_stat.find("div", {"class": "stat-label"}).text.strip())
  print(html_stat.find("div", {"class": "stat-value"}).text.strip())

Undergraduate Students
15,735
Graduate Students
10,163
Faculty
2,864
Nobel Laureates
50
Colleges and Schools
17
undergraduate grant aid (FY23)
$407M


In reality, you probably don't just want to print these items. You want to store them! Let's use **list comprehension** to create two lists, `labels_l` and `value_l`.

In [17]:
label_l = [stat.find("div", {"class": "stat-label"}).text.strip() for stat in cornell_stats]
value_l = [stat.find("div", {"class": "stat-value"}).text.strip() for stat in cornell_stats]

Now, we can store this as a dataframe.

In [18]:
data = {'Label':label_l, 'Stat':value_l}
cornell_df = pd.DataFrame(data)

In [19]:
cornell_df

,Label,Stat
0,Undergraduate Students,"15,735"
1,Graduate Students,"10,163"
2,Faculty,"2,864"
3,Nobel Laureates,50
4,Colleges and Schools,17
5,undergraduate grant aid (FY23),$407M


**Is there any other data cleaning you'd do to this dataframe if you wanted to use it for data science?**

## Collecting text
Often, when you're scraping a website you aren't just collecting headers and tables. You're also collecting text! Now, you're going to collect the text below "Our Profile." Use ^F to find the first sentence in the HTML.

You should find that the text is in a `p` tag below a `div` tag. **What class would you use to extract this `div` tag?**

In [20]:
about_profile = page.find("div", {"class":""}) # THIS CELL WILL NOT EXECUTE UNTIL YOU FILL IN THE CLASS
print(about_profile)

<div id="bN015htcoyT__google-cache-hdr"><div><span>This is Google's cache of <a href="https://www.cornell.edu/about/">https://www.cornell.edu/about/</a>.</span> <span>It is a snapshot of the page as it appeared on Sep 5, 2023 19:10:30 GMT.</span> <span>The <a href="https://www.cornell.edu/about/">current page</a> could have changed in the meantime.</span> <a href="http://support.google.com/websearch/bin/answer.py?hl=en&amp;p=cached&amp;answer=1687222"><span>Learn more</span>.</a></div><div><span style="display:inline-block;margin-top:8px;margin-right:104px;white-space:nowrap"><span style="margin-right:28px"><span style="font-weight:bold">Full version</span></span><span style="margin-right:28px"><a href="http://webcache.googleusercontent.com/search?q=cache:FwQY78IHD64J:https://www.cornell.edu/about/&amp;sca_esv=562865996&amp;hl=en&amp;gl=us&amp;strip=1&amp;vwsrc=0"><span>Text-only version</span></a></span><span style="margin-right:28px"><a href="http://webcache.googleusercontent.com/sea

Before moving on, **do you notice anything odd about the above HTML?**

Now, if we want to extract the natural text from the `about_profile` variable, we can use the `p` tag.

In [21]:
print(about_profile.find('p'))

None


---
# Problem 2: Recreating the timeline as a table

Earlier, you found links to fact pages about Cornell (stored in `link_carets`). One of those links was to a Cornell Timeline (https://www.cornell.edu/about/timeline/). Take some time to inspect the HTML of the page, and then load the cached version with the link below, create a `BeautifulSoup` object, and find some information about Cornell.

Notice the table of data **Cornell Through the Years** with information about Cornell's history. It looks like each event includes three types of information:

1.   Event year
2.   Event title
3.   Event summary

Use `BeautifulSoup` and `requests` to scrape this data and format it as a data frame with one column for the year, another for the title, and a third for the summary.

1. Use the requests library to `get` the website's HTML. Check the status code.

In [22]:
timeline_url = "https://koenecke.infosci.cornell.edu/files/info2950/About%20Cornell%20University%20Timeline.html"

In [23]:
# your code here
timeline_result = requests.get(timeline_url)
if timeline_result.status_code != 200:
  print("something went wrong:", timeline_result.status_code, timeline_result.reason)

2. Convert what you retrieved with the `requests` library to a text string.




In [24]:
# your code here
with open("timeline_about.html", "w") as writer:
  writer.write(timeline_result.text)

with open("timeline_about.html", "r") as reader:
  html_source = reader.read()

3. Parse the text as HTML with BeautifulSoup.

In [25]:
# your code here
page = BeautifulSoup(html_source, "html.parser")

4. Identify the HTML tag and class above the year, title, and summary text. Find all occurences of this tag and save it to a list.

In [26]:
# your code here
contents = page.find_all("div", {"class": "item-content"})
print("there are", len(contents), "links on the page")
contents[:20]

there are 108 links on the page


[<div class="item-content">
 <div class="item-year">1862</div>
 <div class="item-title">
 <h3>A mission to serve New York state</h3>
 </div>
 <div class="item-desc">
 <p></p><div>The
  Morrill Land-Grant Act allows the federal government to appropriate 
 public land, much of which had been expropriated from Indigenous Nations
  across the U.S., to develop and support public colleges. Three years 
 later, Cornell is chartered as  <a href="https://landgrant.cornell.edu/">New York’s land-grant university</a> and charged with improving the lives and livelihoods of the state’s citizens through teaching, research and public service.</div><p></p>
 </div>
 <div class="item-categories" role="group">
 <a class="filter-button-js category-button tag-15" href="https://www.cornell.edu/about/timeline/item-list" role="button">Milestones</a>
 <a class="filter-button-js category-button tag-19" href="https://www.cornell.edu/about/timeline/item-list" role="button">Any Person Any Study</a>
 </div>
 </div>,

5. Access the year, title, and summary text at the 10th index position. Print these.

In [27]:
# your code here
example = contents[9]
example_year = example.find("div", {"class" : "item-year"}).text.strip()
print(example_year)
example_title = example.find("div", {"class" : "item-title"}).text.strip()
print(example_title)
example_summary = example.find("div", {"class" : "item-desc"}).text.strip()
print(example_summary)

1880
Preston is first female doctoral graduate
American educator and suffragist May Preston Slosson
 Ph.D. 1880 becomes the first woman in the country to earn a Ph.D. in 
philosophy. She is also Cornell's first female doctoral graduate.


6. Collect all years, titles, and summaries in the list of timeline events. Add each of these to a list (`year_l`, `title_l`, and `summary_l`).

In [28]:
# your code here
year_l = [c.find("div", {"class": "item-year"}).text.strip() for c in contents]
title_l = [c.find("div", {"class": "item-title"}).text.strip() for c in contents]
summary_l = [c.find("div", {"class": "item-desc"}).text.strip() for c in contents]

7. Convert these lists into data frame columns and display the data frame.

*Note: you might notice a couple of cells in the first and last row have missing information. This is an issue with the cached version of the webpage. You do not need to fix this.*

In [29]:
# your code here
data = {'Year' : year_l, 'Title' : title_l, 'Summary' : summary_l}
timeline_df = pd.DataFrame(data)
timeline_df.head()

,Year,Title,Summary
0,1862,A mission to serve New York state,The\n Morrill Land-Grant Act allows the federa...
1,1865,Cornell charter signed into law,"On April 27, state senators Andrew Dickson Whi..."
2,1868,Cornell chimes ring for the first time,The original set of nine bells first rings out...
3,1868,Cornell University opens,Initially\n divided into two parts — the Divis...
4,1870,Sibley funds creation of engineering college,"Hiram \nSibley, one of the 10 incorporators of..."


---

## Problem 3

Select a page from the web that contains some table element. Extract the table's information and put it into a pandas data frame. Make sure to display the table when you are finished.

As a suggestion, you may use the [Wikipedia page for Cornell University](https://en.wikipedia.org/wiki/Cornell_University). The table on the right side of the page is a good option for scraping and displaying.

Remember to use the following steps, as you did above:
1. Use the requests library to `get` the website's HTML. Check the status code.
2. Convert what you retrieved with the `requests` library to a text string.
3. Parse the text as HTML with BeautifulSoup.
4. Manually search the HTML (with ^F) for the text strings you want to collect. Identify the tags and classes associated with that text.
5. Use combinations of `find` and `find_all` to retrieve the text you are interested in.
6. Convert the text to a data frame. Display this.



In [37]:
wiki_url = "https://koenecke.infosci.cornell.edu/files/info2950/Cornell%20University%20-%20Wikipedia.html" # optional cached version

In [38]:
# your code here, add as many cells as you need
cornell_result = requests.get(cornell_url)
if cornell_result.status_code != 200:
  print("something went wrong:", cornell_result.status_code, cornell_result.reason)


In [39]:
with open("cornell_about.html", "w") as writer:
  writer.write(cornell_result.text)


In [47]:
with open("cornell_about.html", "r") as reader:
  html_source = reader.read()

In [50]:
page = BeautifulSoup(html_source, "html.parser")

In [49]:
infoboxes = page.find_all("h2", {"class": "submenu-title"})
print(infoboxes)

[<h2 class="submenu-title">Related Links</h2>, <h2 class="submenu-title">Undergraduate</h2>, <h2 class="submenu-title">Graduate</h2>, <h2 class="submenu-title">Continuing Education &amp; Summer Sessions</h2>, <h2 class="submenu-title">Related Links</h2>, <h2 class="submenu-title">Explore Cornell Research</h2>, <h2 class="submenu-title">Related Links</h2>, <h2 class="submenu-title">Featured</h2>, <h2 class="submenu-title">Related Links</h2>, <h2 class="submenu-title">Related Links</h2>]


In [53]:
infobox_text = [infobox.text for infobox in infoboxes]
infobox_text

['Related Links',
 'Undergraduate',
 'Graduate',
 'Continuing Education & Summer Sessions',
 'Related Links',
 'Explore Cornell Research',
 'Related Links',
 'Featured',
 'Related Links',
 'Related Links']

In [54]:
data = {"contents" : infobox_text}
df = pd.DataFrame(data)
df.head()

,contents
0,Related Links
1,Undergraduate
2,Graduate
3,Continuing Education & Summer Sessions
4,Related Links


## Submission
**Make sure to complete and save this notebook before next Friday's section.** We will practice pushing and pulling code from GitHub and we will ask you to push a completed version of this notebook to that repository.